In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/text-chunks/gnostic_chunks.json
/kaggle/input/text-chunks/bible_chunks.json


In [2]:
import json

with open("/kaggle/input/text-chunks/bible_chunks.json","r") as f:
    bible = json.load(f)

with open("/kaggle/input/text-chunks/gnostic_chunks.json","r") as f:
    gnostic = json.load(f)

In [4]:
mat = bible['Matthew']
got = gnostic['Gospel of Thomas']

In [11]:
from sentence_transformers import SentenceTransformer,util

model = SentenceTransformer("all-MiniLM-L6-v2",device='cuda')
def embed(text):
    embedding = model.encode(text,show_progress=True, convert_to_numpy=True)
    return embedding

In [10]:
matemb = embed(mat)
gotemb = embed(got)

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

In [15]:
cos = util.cos_sim(gotemb,matemb)

In [17]:
print(gotemb.shape,matemb.shape,cos.shape)

(315, 384) (165, 384) torch.Size([315, 165])


In [24]:
import torch

topsc,topin = torch.topk(cos,k=1)
botsc,botin = torch.topk(-cos,k=1)

In [25]:
topin.shape

torch.Size([315, 1])

In [32]:
i=5
idx = topin[i].item()
sc = topsc[i].item()

g = got[i]
m = mat[idx]

print(g)
print(m)
print(sc)

(3) Jesus said, "If those who lead you say to you, 'See, the kingdom is in the sky,' then the birds of the sky will precede you.
How often have I desired to gather your children together as a hen gathers her brood under her wings, and you were not willing!See, your house is left to you, desolate.For I tell you, you will not see me again until you say, ‘Blessed is the one who comes in the name of the Lord.’ ”As Jesus came out of the temple and was going away, his disciples came to point out to him the buildings of the temple.Then he asked them, “You see all these, do you not?
0.5447791814804077


In [31]:
i=5
idx = botin[i].item()
sc = -botsc[i].item()

g = got[i]
m = mat[idx]

print(g)
print(m)
print(sc)

(3) Jesus said, "If those who lead you say to you, 'See, the kingdom is in the sky,' then the birds of the sky will precede you.
Are you not of more value than they?And which of you by worrying can add a single hour to your span of life?And why do you worry about clothing?
0.02544465661048889


In [33]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

stop_words = set(stopwords.words('english'))

def get_keywords(text):
    tokens = nltk.word_tokenize(text.lower())
    return set([t for t in tokens if t.isalpha() and t not in stop_words])

kws1 = get_keywords(m)
kws2 = get_keywords(g)
keyword_overlap = len(kws1 & kws2) / len(kws1 | kws2)

In [34]:
keyword_overlap

0.058823529411764705